## 1. Get data

In [1]:
import os
import requests
import zipfile
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find directory {image_path}. Creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    with open(data_path/"pizza_steak_sushi.zip","wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading data...")
        f.write(request.content)
    with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip","r") as zip_ref:
        print("unzipping data...")
        zip_ref.extractall(image_path)
    os.remove(data_path/"pizza_steak_sushi.zip")

data\pizza_steak_sushi directory exists.


## 2. Create Datasets and DataLoaders (`data_setup.py`)

In [2]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for image classification data.
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloader(train_dir: str,
                      test_dir: str,
                      transform: transforms.Compose,
                      batch_size: int,
                      num_workers: int = NUM_WORKERS):
    """Creates training and testing DataLoaders.
    Takes in a training directory and testing directory path and turns them into PyTorch Datasets and then into PyTorch DataLoaders.
    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: torchvision transforms to perform on training and testing data
        batch_size: number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader
    Returns:
        A Tuple of (train_dataloader, test_dataloader, class_names)
        Where class_names is a list of target classes.
    Example usage:
        train_dataloader, test_dataloader, class_names = create_dataloader(train_dir=path/to/train_dir,
                                                                           test_dir=path/to/test_dir,
                                                                           transform=transforms.ToTensor(),
                                                                           batch_size=32,
                                                                           num_workers=4)
    """
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    class_names = train_data.classes
    train_dataloader = DataLoader(train_data,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True)
    test_dataloader = DataLoader(test_data,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_workers,
                                 pin_memory=True)
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [3]:
from going_modular import data_setup
from torchvision import transforms
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(image_path/"train",
                                                                               image_path/"test",
                                                                               transform=transforms.ToTensor(),
                                                                               batch_size=32)

## Making a model (`model_builder.py`)

In [4]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch
    Args:
        input_shape: An integer indicating number of input channels
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """
    def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(nn.Conv2d(in_channels=input_shape,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0),
                                          nn.ReLU(),
                                          nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2,
                                                       stride=2))
        self.conv_block_2 = nn.Sequential(nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0),
                                          nn.ReLU(),
                                          nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    stride=1,
                                                    padding=0),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2,
                                                       stride=2))
        self.classifier = nn.Sequential(nn.Flatten(),
                                        nn.Linear(in_features=hidden_units*13*13,
                                                  out_features=output_shape))
    def forward(self,x:torch.Tensor):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting going_modular/model_builder.py


In [5]:
import torch
from going_modular import model_builder
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)

In [6]:
import torch
from going_modular import model_builder
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)